In [1]:
import logging
import pandas as pd
import datetime
import seaborn as sb
import time
from ipywidgets import interact, fixed
from libs.CovidTimeseriesModel import CovidTimeseriesModel
from libs.CovidDatasets import CovidDatasets
pd.set_option('display.float_format', lambda x: '%.12f' % x)

## Constants
start_time = time.time()
HOSPITALIZATION_RATE = .0727
HOSPITALIZED_CASES_REQUIRING_ICU_CARE = .1397
TOTAL_INFECTED_PERIOD = 12
MODEL_INTERVAL = 4
r0 = 2.4
INTERVENTIONS = {
        datetime.date(2020, 3, 23): 1.3,
        datetime.date(2020, 4, 20): 1.1,
        datetime.date(2020, 5, 22): 0.8,
        datetime.date(2020, 6, 23): r0
    }
INTERVENTIONS = None
state = 'CA'
country = 'USA'
Dataset = CovidDatasets()
# Pack all of the assumptions and parameters into a dict that can be passed into the model
MODEL_PARAMETERS = {
    # Pack the changeable model parameters
    'timeseries': Dataset.get_timeseries_by_country_state(country, state, MODEL_INTERVAL),
    'beds': Dataset.get_beds_by_country_state(country, state),
    'population': Dataset.get_population_by_country_state(country, state),
    'projection_iterations': 25, # Number of iterations into the future to project
    'r0': r0,
    'interventions': INTERVENTIONS,
    'hospitalization_rate': HOSPITALIZATION_RATE,
    'initial_hospitalization_rate': .05,
    'case_fatality_rate': .0109341104294479,
    'hospitalized_cases_requiring_icu_care': HOSPITALIZED_CASES_REQUIRING_ICU_CARE,
    # Assumes that anyone who needs ICU care and doesn't get it dies
    'case_fatality_rate_hospitals_overwhelmed': HOSPITALIZATION_RATE * HOSPITALIZED_CASES_REQUIRING_ICU_CARE,
    'hospital_capacity_change_daily_rate': 1.05,
    'max_hospital_capacity_factor': 2.07,
    'initial_hospital_bed_utilization': .6,
    'model_interval': 4, # In days
    'total_infected_period': 12, # In days
    'rolling_intervals_for_current_infected': int(round(TOTAL_INFECTED_PERIOD / MODEL_INTERVAL, 0)),
}

def run_model(model_parameters):
    return CovidTimeseriesModel().forecast_region(model_parameters)

res = run_model(model_parameters=MODEL_PARAMETERS)

In [2]:
res[['Date', 'Pred. Hosp.', 'Recov. or Died', 'New Inf.', 'Cum. Inf.', 'Cum. Deaths', 'End Susceptible']]

,Date,Pred. Hosp.,Recov. or Died,New Inf.,Cum. Inf.,Cum. Deaths,End Susceptible
0,2020-03-03,0.000000000000,0.000000000000,343.878954607978,nan,nan,39511879.121045395732
1,2020-03-07,80.999295048640,0.000000000000,1114.158116212386,1114.158116212386,12.182327878532,39510764.962929181755
2,2020-03-11,176.991928146122,0.000000000000,2434.551968997545,3548.710085209931,38.801987953781,39508330.410960182548
3,2020-03-15,425.938606893110,0.000000000000,5858.852914623250,9407.562999833182,102.863332712164,39502471.558045558631
4,2020-03-19,896.649384868084,343.878954607978,12333.554124732937,21741.117124566117,237.719775499567,39490138.003920823336
5,2020-03-23,2150.755706125958,1458.037070820364,29583.984953589512,51325.102078155629,561.194333925240,39460554.018967233598
6,2020-03-27,5155.063741557767,3892.589039817909,70908.717215375058,122233.819293530687,1336.518078368644,39389645.301751859486
7,2020-03-31,12333.771184955029,9751.441954441159,169652.973658253497,291886.792951784213,3191.522427032203,39219992.328093610704
8,2020-04-04,29382.122767452391,22084.996079174096,404155.746457391884,696042.539409176097,7610.605989493273,38815836.581636212766
9,2020-04-08,69274.260317654393,51668.981032763608,952878.408771037008,1648920.948180213105,27707.097903208705,37862958.172865182161
